In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

정말 최악의 재생산성

In [6]:
mse_pred = list(np.array(pd.read_csv("~/TSTF/main/resulttf/val/trTFTF_coin_mse_pred.csv", index_col = 0)).reshape(2, -1))
mse_pred2 = list(np.array(pd.read_csv("~/TSTF/main/resulttf/test/trTFTF_coin_mse_pred.csv", index_col = 0)).reshape(2, -1))

mae_pred = list(np.array(pd.read_csv("~/TSTF/main/resulttf/val/trTFTF_coin_mae_pred.csv", index_col = 0)).reshape(2, -1))
mae_pred2 = list(np.array(pd.read_csv("~/TSTF/main/resulttf/test/trTFTF_coin_mae_pred.csv", index_col = 0)).reshape(2, -1))

mase_pred = list(np.array(pd.read_csv("~/TSTF/main/resulttf/val/trTFTF_coin_mase_pred.csv", index_col = 0)).reshape(2, -1))
mase_pred2 = list(np.array(pd.read_csv("~/TSTF/main/resulttf/test/trTFTF_coin_mase_pred.csv", index_col = 0)).reshape(2, -1))

mape_pred = list(np.array(pd.read_csv("~/TSTF/main/resulttf/val/trTFTF_coin_mape_pred.csv", index_col = 0)).reshape(2, -1))
mape_pred2 = list(np.array(pd.read_csv("~/TSTF/main/resulttf/test/trTFTF_coin_mape_pred.csv", index_col = 0)).reshape(2, -1))

smape_pred = list(np.array(pd.read_csv("~/TSTF/main/resulttf/val/trTFTF_coin_smape_pred.csv", index_col = 0)).reshape(2, -1))
smape_pred2 = list(np.array(pd.read_csv("~/TSTF/main/resulttf/test/trTFTF_coin_smape_pred.csv", index_col = 0)).reshape(2, -1))

In [7]:
data = "coin"

In [8]:
## target domain
target_X = pd.read_csv(f"../data/{data}/train_input_7.csv").iloc[:, 1:].values.astype(np.float32)
target_y = pd.read_csv(f"../data/{data}/train_output_7.csv").iloc[:, 1:].values.astype(np.float32)

target_X_val = target_X[-round(target_X.shape[0] * 0.2):, :].astype(np.float32)
target_y_val = target_y[-round(target_y.shape[0] * 0.2):].astype(np.float32)
target_X = target_X[:-round(target_X.shape[0] * 0.2), :].astype(np.float32)
target_y = target_y[:-round(target_y.shape[0] * 0.2)].astype(np.float32)

test_X  = pd.read_csv(f"../data/{data}/val_input_7.csv").iloc[:, 1:].values.astype(np.float32)
test_y  = pd.read_csv(f"../data/{data}/val_output_7.csv").iloc[:, 1:].values.astype(np.float32)

In [9]:
val_preds = {}
test_preds = {}
save_name = ["mse", "mae", "mase", "mape", "smape"]

val_preds["mse"] = mse_pred
val_preds["mae"] = mae_pred
val_preds["MASE"] = mase_pred
val_preds["mape"] = mape_pred
val_preds["SMAPE"] = smape_pred

[[array([70.67488 , 60.834126, 88.53074 , ..., 91.67562 , 44.351486,
         80.8119  ], shape=(3480,)),
  array([98.982834, 58.442112, 83.20896 , ..., 47.035103, 32.766827,
         72.18534 ], shape=(3480,))],
 [array([70.92576 , 54.41178 , 77.14634 , ..., 29.121666, 86.966   ,
         67.39412 ], shape=(3480,)),
  array([34.378143, 46.24005 , 50.15614 , ..., 61.89313 , 16.874332,
         49.436813], shape=(3480,))],
 [array([82.099304, 38.44906 , 59.421623, ..., 22.246902, 79.4792  ,
         19.619884], shape=(3480,)),
  array([91.35076 , 48.7464  , 50.237347, ..., 56.658337, 89.70935 ,
         62.148624], shape=(3480,))],
 [array([-0.02848201, -0.1627755 , -0.3066882 , ...,  0.0714244 ,
         -0.05306334,  0.02820593], shape=(3480,)),
  array([0.8622356 , 0.36364168, 0.35910013, ..., 0.25119007, 0.43934613,
         0.12584054], shape=(3480,))],
 [array([-26.567408, 105.903046,  52.778267, ...,  53.806526,  57.888283,
          59.392628], shape=(3480,)),
  array([ 68.82608

In [18]:
concat_G = np.concatenate(list(val_preds.values()))
fin_pred_G = np.median(concat_G, axis = 0)
print("all (RMSE):", np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_G.flatten())).round(5))
print(f"all (MAE):  {mean_absolute_error(target_y_val.flatten(), fin_pred_G.flatten()):.5f}")

## 어떤 점에서 best?
concat_G = np.concatenate([val_preds[loss] for loss in ["mae", "MASE", "mse"]])
fin_pred_G = np.median(concat_G, axis = 0)
print("best (RMSE):", np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_G.flatten())).round(5))
print(f"best (MAE):  {mean_absolute_error(target_y_val.flatten(), fin_pred_G.flatten()):.5f}")

## 변수별 앙상블 결과
for name in ["mse", "mae", "MASE", "mape", "SMAPE"]:
    concat_G = np.concatenate([np.nan_to_num(np.array(val_preds[name]), nan=0)])
    fin_pred_G = np.median(concat_G, axis=0)
    print(name, "(RMSE):", np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_G.flatten())).round(5))
    print(name, f"(MAE):  {mean_absolute_error(target_y_val.flatten(), fin_pred_G.flatten()):.5f}")

all (RMSE): 29.31602
all (MAE):  21.50396
best (RMSE): 22.71764
best (MAE):  16.05210
mse (RMSE): 30.82467
mse (MAE):  24.65280
mae (RMSE): 25.1985
mae (MAE):  19.25673
MASE (RMSE): 26.72937
MASE (MAE):  20.66004
mape (RMSE): 69.82386
mape (MAE):  67.14908
SMAPE (RMSE): 55.74698
SMAPE (MAE):  43.22006


In [19]:
concat_G = np.concatenate([mape_pred, mase_pred, smape_pred, mse_pred, mae_pred])
fin_pred_G = np.median(concat_G, axis=0)
print('all', np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mae_pred, mase_pred, mse_pred])
fin_pred_G = np.median(concat_G, axis=0)
print('best', np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.nan_to_num(np.array(mse_pred), nan=0)])
fin_pred_G = np.median(concat_G, axis=0)
print('mse', np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.array(mase_pred)])
fin_pred_G = np.median(concat_G, axis=0)
print('mase', np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.nan_to_num(np.array(mae_pred), nan=0)])
fin_pred_G = np.median(concat_G, axis=0)
print('mae', np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.array(mape_pred)])
fin_pred_G = np.median(concat_G, axis=0)
print('mape', np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.array(smape_pred)])
fin_pred_G = np.median(concat_G, axis=0)
print('smape', np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_G.flatten())).round(5))

# 지표별 단독 RMSE 저장
concat_mase = np.concatenate([np.nan_to_num(np.array(mase_pred), nan=0)])
fin_pred_mase = np.median(concat_mase, axis=0)
MASE_rmse = np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_mase.flatten())).round(5)

concat_mape = np.concatenate([np.nan_to_num(np.array(mape_pred), nan=0)])
fin_pred_mape = np.median(concat_mape, axis=0)
MAPE_rmse = np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_mape.flatten())).round(5)

concat_smape = np.concatenate([np.nan_to_num(np.array(smape_pred), nan=0)])
fin_pred_smape = np.median(concat_smape, axis=0)
sMAPE_rmse = np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_smape.flatten())).round(5)

concat_mae = np.concatenate([np.nan_to_num(np.array(mae_pred), nan=0)])
fin_pred_mae = np.median(concat_mae, axis=0)
MAE_rmse = np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_mae.flatten())).round(5)

concat_mse = np.concatenate([np.nan_to_num(np.array(mse_pred), nan=0)])
fin_pred_mse = np.median(concat_mse, axis=0)
MSE_rmse = np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_mse.flatten())).round(5)

all 29.31602
best 22.71764
mse 30.82467
mase 26.72937
mae 25.1985
mape 69.82386
smape 55.74698


In [21]:
performance = np.array([MASE_rmse, MAPE_rmse, sMAPE_rmse, MAE_rmse, MSE_rmse])

In [ ]:
performance = np.array([MASE_rmse, MAPE_rmse, sMAPE_rmse, MAE_rmse, MSE_rmse])
os.makedirs('resulttf', exist_ok=True)
pd.DataFrame(performance).to_csv(f"resulttf/trTFTF_{data}_weight.csv")